# Safe Album Update Notebook

This notebook safely updates album titles and artists while preserving all database relationships.

**What this does:**
- Updates only the `title` and `artist` columns
- Keeps the same `album_id` (no cascade deletions)
- Preserves all relationships (inventory, order_items, reviews)

**Steps:**
1. Preview changes
2. Run update
3. Verify results

## Setup and Imports

In [6]:
import os
import sys
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Add parent directory to import db_connector
sys.path.append('..')

from db_connector import DatabaseConnector

load_dotenv()

print("✅ Imports successful")

✅ Imports successful


## Step 1: Preview Changes

Let's see what will be updated before making any changes.

In [7]:
# Load the CSV files
old_csv = 'albums_rows.csv'
new_csv = 'new.csv'

old_df = pd.read_csv(old_csv)
new_df = pd.read_csv(new_csv)

print(f"📊 Old CSV: {len(old_df)} albums")
print(f"📊 New CSV: {len(new_df)} albums\n")

# Show first few rows
print("Old data sample:")
print(old_df[['album_id', 'title', 'artist']].head(3))
print("\nNew data sample:")
print(new_df[['album_id', 'title', 'artist']].head(3))

📊 Old CSV: 298 albums
📊 New CSV: 298 albums

Old data sample:
                               album_id           title             artist
0  0081c90d-2164-43dc-82dd-5f2899e1d278        The Epic  Kamasi Washington
1  00c06829-aadb-4775-a914-452ecf25659b  A Love Supreme      John Coltrane
2  0181ed08-a6f5-4f83-abe7-d9372310a3ec    Kind of Blue        Miles Davis

New data sample:
                               album_id                   title        artist
0  0081c90d-2164-43dc-82dd-5f2899e1d278               All Blues   Miles Davis
1  00c06829-aadb-4775-a914-452ecf25659b               All of Me  Gerald Marks
2  0181ed08-a6f5-4f83-abe7-d9372310a3ec  All the Things You Are   Jerome Kern


In [8]:
# Compare old vs new to see what's changing
comparison = pd.merge(
    old_df[['album_id', 'title', 'artist']],
    new_df[['album_id', 'title', 'artist']],
    on='album_id',
    suffixes=('_old', '_new')
)

# Find changes
title_changes = comparison[comparison['title_old'] != comparison['title_new']]
artist_changes = comparison[comparison['artist_old'] != comparison['artist_new']]

print(f"📊 Change Summary:")
print(f"   Total albums: {len(new_df)}")
print(f"   Title changes: {len(title_changes)}")
print(f"   Artist changes: {len(artist_changes)}")
print()

# Show some examples
if len(title_changes) > 0:
    print("\n🎵 Sample Title Changes (first 10):")
    for idx, row in title_changes.head(10).iterrows():
        print(f"   '{row['title_old']}' → '{row['title_new']}'")

if len(artist_changes) > 0:
    print("\n🎤 Sample Artist Changes (first 10):")
    for idx, row in artist_changes.head(10).iterrows():
        print(f"   '{row['artist_old']}' → '{row['artist_new']}'")

📊 Change Summary:
   Total albums: 298
   Title changes: 298
   Artist changes: 291


🎵 Sample Title Changes (first 10):
   'The Epic' → 'All Blues'
   'A Love Supreme' → 'All of Me'
   'Kind of Blue' → 'All the Things You Are'
   'Kind of Blue' → 'Alone Together'
   'Kind of Blue' → 'Anthropology'
   'Kind of Blue' → 'Autumn Leaves'
   'Kind of Blue' → 'Beautiful Love'
   'Kind of Blue' → 'Blue Bossa'
   'Kind of Blue' → 'Blue Monk'
   'Kind of Blue' → 'Body and Soul'

🎤 Sample Artist Changes (first 10):
   'Kamasi Washington' → 'Miles Davis'
   'John Coltrane' → 'Gerald Marks'
   'Miles Davis' → 'Jerome Kern'
   'Miles Davis' → 'Arthur Schwartz'
   'Miles Davis' → 'Charlie Parker'
   'Miles Davis' → 'Joseph Kosma'
   'Miles Davis' → 'Victor Young'
   'Miles Davis' → 'Kenny Dorham'
   'Miles Davis' → 'Thelonious Monk'
   'Miles Davis' → 'Johnny Green'


## Step 2: Connect to Database

In [9]:
# Initialize database connection
db = DatabaseConnector()
db.connect()

print("✅ Connected to database")

Connected to Supabase successfully
✅ Connected to database


## Step 3: Perform the Update

⚠️ **This will modify your database!** Make sure you've reviewed the preview above.

In [12]:
# Update albums in batches - ONLY title and artist
BATCH_SIZE = 50
total_updated = 0
total_failed = 0

print(f"🔄 Starting update of {len(new_df)} albums...")
print(f"⚠️  Updating ONLY 'title' and 'artist' columns")
print(f"✅ Preserving: album_id, genre_id, label_id, price, created_at\n")

for i in range(0, len(new_df), BATCH_SIZE):
    batch = new_df.iloc[i:i+BATCH_SIZE]
    print(f"📦 Processing batch {i//BATCH_SIZE + 1} ({len(batch)} records)...")
    
    for idx, row in batch.iterrows():
        try:
            # Update ONLY title and artist - nothing else
            db.client.table('albums').update({
                'title': row['title'],
                'artist': row['artist']
            }).eq('album_id', row['album_id']).execute()
            
            total_updated += 1
            
            # Progress indicator
            if total_updated % 10 == 0:
                print(f"  ✓ Updated {total_updated}/{len(new_df)}...")
        
        except Exception as e:
            total_failed += 1
            print(f"  ❌ Failed to update album {row['album_id']}: {e}")

print(f"\n{'='*60}")
print(f"✅ Update Complete!")
print(f"{'='*60}")
print(f"✓ Successfully updated: {total_updated}")
if total_failed > 0:
    print(f"❌ Failed updates: {total_failed}")
print(f"{'='*60}")


🔄 Starting update of 298 albums...
⚠️  Updating ONLY 'title' and 'artist' columns
✅ Preserving: album_id, genre_id, label_id, price, created_at

📦 Processing batch 1 (50 records)...
  ✓ Updated 10/298...
  ✓ Updated 20/298...
  ✓ Updated 30/298...
  ✓ Updated 40/298...
  ✓ Updated 50/298...
📦 Processing batch 2 (50 records)...
  ✓ Updated 60/298...
  ✓ Updated 70/298...
  ✓ Updated 80/298...
  ✓ Updated 90/298...
  ✓ Updated 100/298...
📦 Processing batch 3 (50 records)...
  ✓ Updated 110/298...
  ✓ Updated 120/298...
  ✓ Updated 130/298...
  ✓ Updated 140/298...
  ✓ Updated 150/298...
📦 Processing batch 4 (50 records)...
  ✓ Updated 160/298...
  ✓ Updated 170/298...
  ✓ Updated 180/298...
  ✓ Updated 190/298...
  ✓ Updated 200/298...
📦 Processing batch 5 (50 records)...
  ✓ Updated 210/298...
  ✓ Updated 220/298...
  ✓ Updated 230/298...
  ✓ Updated 240/298...
  ✓ Updated 250/298...
📦 Processing batch 6 (48 records)...
  ✓ Updated 260/298...
  ✓ Updated 270/298...
  ✓ Updated 280/298..

## Step 4: Verify the Update

Let's check a random sample to make sure the updates worked.

In [13]:
# Verify by checking random samples
sample_size = 10
sample = new_df.sample(min(sample_size, len(new_df)))

print(f"🔍 Verifying {len(sample)} random albums:\n")
all_match = True

for idx, row in sample.iterrows():
    # Fetch from database
    result = db.client.table('albums').select(
        'album_id, title, artist'
    ).eq('album_id', row['album_id']).execute()
    
    if result.data:
        db_record = result.data[0]
        matches = (
            db_record['title'] == row['title'] and
            db_record['artist'] == row['artist']
        )
        
        status = "✅" if matches else "❌"
        print(f"{status} Album ID: {row['album_id'][:8]}...")
        print(f"   CSV:  '{row['title']}' by {row['artist']}")
        print(f"   DB:   '{db_record['title']}' by {db_record['artist']}")
        print()
        
        if not matches:
            all_match = False
    else:
        print(f"❌ Album {row['album_id']} not found in database!")
        all_match = False

if all_match:
    print("\n✅ All sampled records match! Update successful.")
else:
    print("\n⚠️  Some records don't match. Please investigate.")

🔍 Verifying 10 random albums:

✅ Album ID: c9a387cc...
   CSV:  '26-2' by John Coltrane
   DB:   '26-2' by John Coltrane

✅ Album ID: 1de8844d...
   CSV:  'Indiana' by James F. Hanley
   DB:   'Indiana' by James F. Hanley

✅ Album ID: 6ace4a92...
   CSV:  'Cousin Mary' by John Coltrane
   DB:   'Cousin Mary' by John Coltrane

✅ Album ID: 59a65f61...
   CSV:  'You Stepped Out of a Dream' by Nacio Herb Brown
   DB:   'You Stepped Out of a Dream' by Nacio Herb Brown

✅ Album ID: adde367a...
   CSV:  'Skylark' by Hoagy Carmichael
   DB:   'Skylark' by Hoagy Carmichael

✅ Album ID: f84ffdc8...
   CSV:  'Ben' by Walter Scharf
   DB:   'Ben' by Walter Scharf

✅ Album ID: 115ee252...
   CSV:  'Embraceable You' by George Gershwin
   DB:   'Embraceable You' by George Gershwin

✅ Album ID: 2dfdbccd...
   CSV:  'My One and Only Love' by Guy Wood
   DB:   'My One and Only Love' by Guy Wood

✅ Album ID: d621fa51...
   CSV:  'Watch What Happens' by Michel Legrand
   DB:   'Watch What Happens' by Mich

## Step 5: Verify No Cascade Deletions Occurred

Let's confirm that all related data (inventory, order_items, reviews) is still intact.

In [14]:
# Check counts of related tables
print("🔍 Checking related table counts:\n")

# Count inventory records
inventory_count = db.client.table('inventory').select('inventory_id', count='exact').execute()
print(f"📦 Inventory records: {inventory_count.count}")

# Count order items
order_items_count = db.client.table('order_items').select('order_item_id', count='exact').execute()
print(f"🛒 Order items: {order_items_count.count}")

# Count reviews
reviews_count = db.client.table('reviews').select('review_id', count='exact').execute()
print(f"⭐ Reviews: {reviews_count.count}")

# Count albums
albums_count = db.client.table('albums').select('album_id', count='exact').execute()
print(f"\n💿 Total albums: {albums_count.count}")

print("\n✅ If all counts look correct, the update was successful without any cascade deletions!")

🔍 Checking related table counts:

📦 Inventory records: 298
🛒 Order items: 1601
⭐ Reviews: 167

💿 Total albums: 298

✅ If all counts look correct, the update was successful without any cascade deletions!


## Cleanup

In [15]:
# Close database connection
db.close()
print("✅ Database connection closed")

Database connection closed
✅ Database connection closed
